In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
# Basics
import tensorflow as tf
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

<IPython.core.display.Javascript object>

### 0. Set up

In [4]:
lead_time = 0

<IPython.core.display.Javascript object>

### 1. Load Data

In [5]:
dat_train_denorm = ldpd.load_data_all_train_proc_denorm()
dat_test_denorm = ldpd.load_data_all_test_proc_denorm()

<IPython.core.display.Javascript object>

### 2. Data Split

In [6]:
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    dat_train_denorm
)
dat_X_test_lead_all_denorm, dat_y_test_lead_all_denorm = split_var_lead(dat_test_denorm)

<IPython.core.display.Javascript object>

### 3. Data preparation

Idea: X_train constructed by flattening the value arrays and concatenating them together, so that in the end I get the shape (n, n_variables), keep in mind we use both mean and std of each variabel for training. So in total we will have 10 / 12, depending on which variable. Important, for drn we'll also need the embeddings, hence the idea to keep another flattened array with shape (n, 1) with the complementing embedding (1 - 15600).

In [7]:
# Make X train data into NN feedable array
drn_X_train_0_array, drn_embedding_train_0_array = make_X_array(
    dat_X_train_lead_all_denorm, 0
)

<IPython.core.display.Javascript object>

In [8]:
# Make X test data into NN feedable array
drn_X_test_0_array, drn_embedding_test_0_array = make_X_array(
    dat_X_test_lead_all_denorm, 0
)

<IPython.core.display.Javascript object>

In [9]:
# Put y values into correct form
t2m_y_train = dat_y_train_lead_all_denorm[2][0].values.flatten()
t2m_y_test = dat_y_test_lead_all_denorm[2][0].values.flatten()

<IPython.core.display.Javascript object>

### 4. Model SetUp

In [10]:
hidden_layer = []
emb_size = 3
max_id = 15600 - 1

<IPython.core.display.Javascript object>

In [15]:
drn_0_model = build_emb_model(12, 2, hidden_layer, emb_size, max_id, compile=True)

<IPython.core.display.Javascript object>

### 5. Train DRN for lead_time 0, t2m

In [16]:
drn_0_model.fit(
    [drn_X_train_0_array, drn_embedding_train_0_array],
    t2m_y_train,
    epochs=10,
    batch_size=8192,
    validation_split=0.2,
)

Epoch 1/10
2177/2177 [==============================] - 9s 3ms/step - loss: 62.4298 - val_loss: 2.7839
Epoch 2/10
2177/2177 [==============================] - 7s 3ms/step - loss: 2.7070 - val_loss: 2.5978
Epoch 3/10
2177/2177 [==============================] - 7s 3ms/step - loss: 2.5247 - val_loss: 2.3995
Epoch 4/10
2177/2177 [==============================] - 7s 3ms/step - loss: 2.2923 - val_loss: 2.1264
Epoch 5/10
2177/2177 [==============================] - 7s 3ms/step - loss: 2.0252 - val_loss: 1.8559
Epoch 6/10
2177/2177 [==============================] - 7s 3ms/step - loss: 1.7622 - val_loss: 1.6266
Epoch 7/10
2177/2177 [==============================] - 7s 3ms/step - loss: 1.5433 - val_loss: 1.4343
Epoch 8/10
2177/2177 [==============================] - 7s 3ms/step - loss: 1.3716 - val_loss: 1.2840
Epoch 9/10
2177/2177 [==============================] - 7s 3ms/step - loss: 1.2439 - val_loss: 1.1724
Epoch 10/10
2177/2177 [==============================] - 7s 3ms/step - loss: 1.15

<IPython.core.display.Javascript object>

In [17]:
drn_0_preds = drn_0_model.predict(
    [drn_X_test_0_array, drn_embedding_test_0_array], verbose=1
)

174038/174038 [==============================] - 117s 669us/step


<IPython.core.display.Javascript object>

In [22]:
drn_0_score = crps_normal(mu=drn_0_preds[:, 0], sigma=drn_0_preds[:, 1], y=t2m_y_test)

<IPython.core.display.Javascript object>

In [28]:
drn_0_score.shape

(5569200,)

<IPython.core.display.Javascript object>

In [19]:
drn_0_preds

array([[278.9877   ,   1.5516199],
       [279.61688  ,   1.4355881],
       [279.0234   ,   1.4504821],
       ...,
       [290.25305  ,   1.7038431],
       [290.3067   ,   1.6643829],
       [291.057    ,   1.6139333]], dtype=float32)

<IPython.core.display.Javascript object>

In [20]:
EMOS_global_t2m_score = EMOS_global_load_score("t2m")

<IPython.core.display.Javascript object>

In [21]:
EMOS_global_t2m_score[0].mean()

0.4673401873320892

<IPython.core.display.Javascript object>